<a href="https://colab.research.google.com/github/nivedanyakolli361/CNN1/blob/main/Models/Functional_API_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Part 1 - Building the CNN
# Multiple Inputs

from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [4]:
Train = ImageDataGenerator(rescale = 1./255,
                           rotation_range = 20,
                           shear_range = 0.2,
                           zoom_range = 0.5,
                           horizontal_flip = True,
                           vertical_flip= True)

Test = ImageDataGenerator(rescale = 1./255)

In [11]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import PIL 

# import required module
import glob

# get the path/directory
folder_dir = '/content/drive/MyDrive/OKC_and_NonOKC_Dataset/Test/OKC'

# iterate over files in that directory
num = 0
for images in glob.iglob(f'{folder_dir}/*.jpg'):
  img = cv2.imread(images)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  canny = cv2.Canny(img, 100, 200)
  canny = PIL.Image.fromarray(canny)
  name = '/content/drive/MyDrive/Canny/Test/OKC/OKC_'+str(num)+'.jpg'
  canny.save(name,'JPEG')
  num+=1

In [13]:
# Pythono3 code to rename multiple files in a directory or folder

# importing os module
import os

# Function to rename multiple files
def main():
    folder = "/content/drive/MyDrive/Canny/Training/OKC"
    for count, filename in enumerate(os.listdir(folder)):
        dst = f"OKC_{str(count)}.jpg"
        src =f"{folder}/{filename}" # foldername/filename, if .py file is outside folder
        dst =f"{folder}/{dst}"

        # rename() function will
        # rename all the files
        os.rename(src, dst)

# Driver Code
if __name__ == '__main__':

    # Calling main() function
    main()

In [15]:
t1 = Train.flow_from_directory('/content/drive/MyDrive/OKC_and_NonOKC_Dataset/Training',
                               target_size = (512, 512),
                               batch_size=2,
                               class_mode='binary',
                               shuffle = True, 
                               seed = 13, 
                               color_mode = 'rgb')
t2 = Train.flow_from_directory('/content/drive/MyDrive/Canny/Training',
                               target_size = (512, 512),
                               batch_size=2,
                               class_mode='binary',
                               shuffle = True, 
                               seed= 13, 
                               color_mode = 'grayscale')

Found 2003 images belonging to 2 classes.
Found 2003 images belonging to 2 classes.


In [16]:
import tensorflow as tf 
device__list = tf.test.gpu_device_name()
device__list

'/device:GPU:0'

In [17]:
Validation = ImageDataGenerator(rescale = 1./255)

In [18]:
validation_dataset1 = Validation.flow_from_directory('/content/drive/MyDrive/OKC_and_NonOKC_Dataset/Validation',
                                                     target_size = (512, 512),
                                                     batch_size=1,
                                                     class_mode='binary',
                                                     shuffle = False, 
                                                     seed = 13)

validation_dataset2 = Validation.flow_from_directory('/content/drive/MyDrive/Canny/Validation',
                                                     target_size = (512, 512),
                                                     batch_size=1,
                                                     class_mode='binary',
                                                     shuffle = False, 
                                                     seed = 13)


test_dataset1 = Test.flow_from_directory('/content/drive/MyDrive/OKC_and_NonOKC_Dataset/Test',
                                         target_size = (512, 512),
                                         batch_size = 1,
                                         class_mode = 'binary',
                                         shuffle = False, 
                                         seed = 13)

test_dataset2 = Test.flow_from_directory('/content/drive/MyDrive/Canny/Test',
                                         target_size = (512, 512),
                                         batch_size = 1,
                                         class_mode = 'binary',
                                         shuffle = False, 
                                         seed = 13)

Found 430 images belonging to 2 classes.
Found 430 images belonging to 2 classes.
Found 428 images belonging to 2 classes.
Found 428 images belonging to 2 classes.


In [19]:
# first input model
visible1 = Input(shape=(512,512,1))
conv11 = Conv2D(32, kernel_size=4, activation='relu')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)

conv12 = Conv2D(16, kernel_size=4, activation='relu')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)


# second input model
visible2 = Input(shape=(512,512,3))
conv21 = Conv2D(32, kernel_size=4, activation='relu')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)

conv22 = Conv2D(16, kernel_size=4, activation='relu')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)


# merge input models
merge = concatenate([flat1, flat2])


# interpretation model
hidden1 = Dense(512, activation='relu')(merge)
hidden2 = Dense(128, activation='relu')(hidden1)

output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=[visible1, visible2], outputs=output)

In [20]:
# summarize layers
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 509, 509, 32  544         ['input_3[0][0]']                
                                )                                                           

In [21]:
def generator_traing():
  while True:
    train1 = t1.next()
    train2 = t2.next()
    yield ([train1[0], train2[0]], train1[1])

genator_1 = generator_traing()   #for training   

In [22]:
def generator_validation():
  while True:
    train_val1 = validation_dataset1.next()
    train_val2 = validation_dataset2.next()
    yield ([train_val1[0], train_val2[0]], train_val1[1])

genator_2 = generator_validation()

In [23]:
def generator_test():
  while True:
    train_test1 = test_dataset1.next()
    train_test2 = test_dataset2.next()
    yield ([train_test1[0], train_test2[0]], train_test1[1])

genator_3 = generator_validation()

In [24]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [28]:
history = model.fit(genator_1,
                    validation_data = genator_3, 
                    epochs = 10,
                    steps_per_epoch=1002,
                    validation_steps=230)

Epoch 1/10


InvalidArgumentError: ignored